In [1]:
import sys
sys.path.append('../modules')
from aggregate_to_healpix import aggregate_to_healpix
from lonlat_to_healpix import add_healpix_to_dataarray

%load_ext autoreload
%autoreload 2


# Aggregate DataArray to HEALPix

This notebook demonstrates the simplest aggregation method for converting lon/lat gridded data to HEALPix format.

**Key concept**: Multiple pixels mapping to the same HEALPix cell are aggregated using mean.

**Method**: Based on EOPF examples:
- Stack spatial dimensions → `cells` dimension
- Assign HEALPix cell IDs as coordinate
- Group by cell IDs and compute mean
- Result: One value per unique HEALPix cell


In [2]:
# Create a small test DataArray with lon/lat coordinates
import numpy as np
import xarray as xr

# Create a small 3x3 grid around Biscay location
n_rows, n_cols = 3, 3
center_lon, center_lat = -4.5, 48.0  # Biscay coordinates

# Create a regular grid
lon_spacing = 0.01  # degrees
lat_spacing = 0.01  # degrees

lon = np.linspace(center_lon - (n_cols-1)*lon_spacing/2, 
                  center_lon + (n_cols-1)*lon_spacing/2, n_cols)
lat = np.linspace(center_lat - (n_rows-1)*lat_spacing/2, 
                  center_lat + (n_rows-1)*lat_spacing/2, n_rows)

lon_2d, lat_2d = np.meshgrid(lon, lat)

# Create some dummy data
test_data = np.round(np.random.rand(n_rows, n_cols), 4)

# Create DataArray with lon/lat coordinates
test_da = xr.DataArray(
    test_data,
    dims=['rows', 'columns'],
    coords={
        'longitude': (['rows', 'columns'], lon_2d),
        'latitude': (['rows', 'columns'], lat_2d),
    },
    name='test_variable'
)

print("Created test DataArray:")
print(f"  Shape: {test_da.shape}")
print(f"  Lon range: {lon_2d.min():.4f} to {lon_2d.max():.4f}")
print(f"  Lat range: {lat_2d.min():.4f} to {lat_2d.max():.4f}")
print(f"\nOriginal data values:")
print(test_da.values)
test_da


Created test DataArray:
  Shape: (3, 3)
  Lon range: -4.5100 to -4.4900
  Lat range: 47.9900 to 48.0100

Original data values:
[[0.8792 0.0523 0.8066]
 [0.3698 0.8996 0.4446]
 [0.4702 0.5916 0.5293]]


<xarray.DataArray 'test_variable' (rows: 3, columns: 3)> Size: 72B
array([[0.8792, 0.0523, 0.8066],
       [0.3698, 0.8996, 0.4446],
       [0.4702, 0.5916, 0.5293]])
Coordinates:
    longitude  (rows, columns) float64 72B -4.51 -4.5 -4.49 ... -4.51 -4.5 -4.49
    latitude   (rows, columns) float64 72B 47.99 47.99 47.99 ... 48.01 48.01
Dimensions without coordinates: rows, columns

## Step 1: Add HEALPix cell IDs (tagging)

First, we add HEALPix cell IDs to each pixel. This shows which HEALPix cell each pixel belongs to, but doesn't aggregate yet.


In [3]:
# Add HEALPix cell IDs to the DataArray (tagging only, no aggregation)
test_da_tagged = add_healpix_to_dataarray(test_da, verbose=True)

print("\n" + "="*60)
print("Tagged DataArray (before aggregation):")
print("="*60)
print(f"  Shape: {test_da_tagged.shape}")
print(f"  Dimensions: {test_da_tagged.dims}")
print(f"\n  HEALPix cell IDs:")
print(test_da_tagged.coords['healpix'].values)
print(f"\n  Unique cells: {len(np.unique(test_da_tagged.coords['healpix'].values[~np.isnan(test_da_tagged.coords['healpix'].values)]))}")
print(f"\n  Data values (still on original grid):")
print(test_da_tagged.values)


Calculating pixel sizes...
PIXEL SIZE STATISTICS

Row-wise pixel sizes (North-South direction):
  Mean:   1111.90 m
  Median: 1111.90 m
  Min:    1111.90 m
  Max:    1111.90 m

Column-wise pixel sizes (East-West direction):
  Mean:   746.25 m
  Median: 746.25 m
  Min:    746.11 m
  Max:    746.40 m

Using pixel size: 1111.90 m (max of row/column)

Selected HEALPix level: 12
  Pixel size: 1111.90 m
  Edge length: 1591.71 m
  Ratio: 1.43x

Calculating HEALPix cell IDs at level 12...
Added HEALPix coordinates:
  Valid pixels: 9 / 9
  Unique cells: 5
  Level: 12
  Grid mapping: crs (CF-compliant)

Tagged DataArray (before aggregation):
  Shape: (3, 3)
  Dimensions: ('rows', 'columns')

  HEALPix cell IDs:
[[56464103 56464103 56464114]
 [56464109 56464109 56464114]
 [56464111 56464111 56464120]]

  Unique cells: 5

  Data values (still on original grid):
[[0.8792 0.0523 0.8066]
 [0.3698 0.8996 0.4446]
 [0.4702 0.5916 0.5293]]


/home/kajetan/miniconda3/envs/healpix/lib/python3.13/site-packages/numpy/_core/numeric.py:387: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


## Step 2: Aggregate to HEALPix cells

Now we aggregate: pixels mapping to the same HEALPix cell are averaged. The result has one value per unique HEALPix cell.


In [4]:
# Aggregate to HEALPix cells using grouped mean
test_da_aggregated = aggregate_to_healpix(test_da_tagged, verbose=True)

print("\n" + "="*60)
print("Aggregated DataArray (after aggregation):")
print("="*60)
print(f"  Shape: {test_da_aggregated.shape}")
print(f"  Dimensions: {test_da_aggregated.dims}")
print(f"\n  Cell IDs: {test_da_aggregated.coords['cell_ids'].values}")
print(f"  Aggregated values: {test_da_aggregated.values}")
print(f"\n  Number of cells: {len(test_da_aggregated.coords['cell_ids'])}")
print(f"  (Reduced from {test_da.shape[0] * test_da.shape[1]} pixels)")

test_da_aggregated


Aggregating to HEALPix level 12...
Stacking dimensions ('rows', 'columns') into 'cells'...


Grouping 9 pixels into 5 unique cells...
Aggregation complete: 5 unique HEALPix cells

Aggregated DataArray (after aggregation):
  Shape: (5,)
  Dimensions: ('cells',)

  Cell IDs: [56464103 56464109 56464111 56464114 56464120]
  Aggregated values: [0.46575 0.6347  0.5309  0.6256  0.5293 ]

  Number of cells: 5
  (Reduced from 9 pixels)


<xarray.DataArray 'test_variable' (cells: 5)> Size: 40B
array([0.46575, 0.6347 , 0.5309 , 0.6256 , 0.5293 ])
Coordinates:
  * cells     (cells) int64 40B 56464103 56464109 56464111 56464114 56464120
    crs       int8 1B 0
    cell_ids  (cells) int64 40B 56464103 56464109 56464111 56464114 56464120
Attributes:
    coordinates:   healpix
    grid_mapping:  crs

## Comparison: Before vs After Aggregation

**Before aggregation** (tagging only):
- Shape: `(rows, columns)` = `(3, 3)` = 9 pixels
- Each pixel has its own value
- Multiple pixels may share the same HEALPix cell ID

**After aggregation**:
- Shape: `(cells,)` = `(N,)` where N = number of unique HEALPix cells
- One value per unique HEALPix cell
- Value is the mean of all pixels that mapped to that cell


In [5]:
# Verify aggregation: show which pixels map to which cells
print("Pixel-to-cell mapping:")
print("="*60)
cell_ids_2d = test_da_tagged.coords['healpix'].values
data_2d = test_da_tagged.values

for i in range(n_rows):
    for j in range(n_cols):
        cell_id = cell_ids_2d[i, j]
        value = data_2d[i, j]
        if not np.isnan(cell_id):
            print(f"  Pixel ({i},{j}): value={value:.4f} → cell {int(cell_id)}")

print("\nAggregated result:")
print("="*60)
for cell_id, value in zip(test_da_aggregated.coords['cell_ids'].values, 
                          test_da_aggregated.values):
    # Find all pixels that map to this cell
    pixels_in_cell = np.where(cell_ids_2d == cell_id)
    pixel_values = [data_2d[i, j] for i, j in zip(pixels_in_cell[0], pixels_in_cell[1])]
    mean_value = np.mean(pixel_values)
    print(f"  Cell {cell_id}: mean({pixel_values}) = {mean_value:.4f} (aggregated: {value:.4f})")


Pixel-to-cell mapping:
  Pixel (0,0): value=0.8792 → cell 56464103
  Pixel (0,1): value=0.0523 → cell 56464103
  Pixel (0,2): value=0.8066 → cell 56464114
  Pixel (1,0): value=0.3698 → cell 56464109
  Pixel (1,1): value=0.8996 → cell 56464109
  Pixel (1,2): value=0.4446 → cell 56464114
  Pixel (2,0): value=0.4702 → cell 56464111
  Pixel (2,1): value=0.5916 → cell 56464111
  Pixel (2,2): value=0.5293 → cell 56464120

Aggregated result:
  Cell 56464103: mean([np.float64(0.8792), np.float64(0.0523)]) = 0.4657 (aggregated: 0.4657)
  Cell 56464109: mean([np.float64(0.3698), np.float64(0.8996)]) = 0.6347 (aggregated: 0.6347)
  Cell 56464111: mean([np.float64(0.4702), np.float64(0.5916)]) = 0.5309 (aggregated: 0.5309)
  Cell 56464114: mean([np.float64(0.8066), np.float64(0.4446)]) = 0.6256 (aggregated: 0.6256)
  Cell 56464120: mean([np.float64(0.5293)]) = 0.5293 (aggregated: 0.5293)
